# Embedded Topic Model (ETM)

In [1]:
import spacy
import string
from octis.preprocessing.preprocessing import Preprocessing
from octis.models.ETM import ETM
from octis.dataset.dataset import Dataset
from octis.evaluation_metrics.diversity_metrics import TopicDiversity
from octis.evaluation_metrics.coherence_metrics import Coherence
from octis.optimization.optimizer import Optimizer
from skopt.space.space import Real
import torch
import numpy as np
import pandas as pd
import os
from wordcloud import WordCloud

In [2]:

current_directory = os.getcwd()
parent_directory = os.path.dirname(current_directory)
os.chdir(parent_directory)

In [3]:
from utils.embeddings import *
from preprocessing.clean_text import *

In [4]:
file_path = 'data/raw/cleaned_train_lyrics.csv'
df = pd.read_csv(file_path)
df = df.drop(columns = ['Unnamed: 0'])
df = df.rename(columns = {'Lyric':'lyrics'})
df = df.sample(frac=0.0001).reset_index(drop=True)

## Embeddings

In [5]:
embeddings_file = 'data/input/embeddings.pkl'

In [6]:
# get first value of the metadata, containing the number of documents form dict


In [7]:
embeddings = create_embeddings(df, embeddings_file, force_creation=True, batch_size=64)

Embeddings not found, generating...


c:\Users\marta\miniconda3\envs\OCTIS\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


cpu
Embeddings saved to data/input/embeddings.pkl


## Preprocessing


In [8]:
# Setting up the preprocessing 
nlp = spacy.load('en_core_web_sm')


In [9]:
extract_corpus_and_labels_from_songs_csv(csv_input_path = 'data/raw/cleaned_train_lyrics.csv', output_path = 'data/input',frac=0.0001)

Corpus has been saved to data/input\corpus.txt
Labels have been saved to data/input\labels.txt


In [10]:
stopwords_list = list(nlp.Defaults.stop_words)
stopwords_list.extend(CUSTOM_STOPWORDS)

In [11]:
preprocessor = Preprocessing(lowercase=True,
                             min_df = 10,
                             max_df = 0.85,
                             remove_punctuation=True,
                             punctuation=string.punctuation,
                             remove_numbers=True,
                             lemmatize= True,
                             stopword_list=stopwords_list,
                             min_chars=3,
                             min_words_docs=10,
                             language='english',
                             split=True,
                             verbose=True)

In [12]:

# Checks if dataset folder exists in processed, if not it processes the dataset. Otherwise it loads it
if not os.path.exists('data/processed/dataset'):
    dataset = preprocessor.preprocess_dataset(documents_path = 'data/input/corpus.txt',labels_path = 'data/input/labels.txt')
    dataset.save('data/processed/dataset')
else:
    dataset = Dataset()
    dataset.load_custom_dataset_from_folder('data/processed/dataset')

In [13]:
dataset.get_metadata()

{'total_documents': 50,
 'vocabulary_length': 20,
 'preprocessing-info': ['lowercase',
  'remove_punctuation',
  'lemmatize',
  'filter words with document frequency lower than 10 and higher than 0.85',
  'filter words with less than 3 character',
  'filter documents with less than 10 words'],
 'last-training-doc': 15,
 'last-validation-doc': 19}

## ETM model without optimization

In [14]:
N_TOPICS = 5

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [15]:
#model = ETM(num_topics = 10, vocab_size=3000, t_hidden_size=800, theta_act = 'relu', embeddings = None, train_embeddings = True, enc_drop = 0.0, rho_size= 5, emb_size= 10)7


In [16]:
model = ETM(num_topics= N_TOPICS,  
        num_epochs=100, 
        t_hidden_size=800, 
        rho_size=384, 
        embedding_size=384, 
        activation='relu', 
        dropout=0.5, 
        lr=0.005, 
        optimizer='adam', 
        batch_size=128, 
        clip=0.0, 
        wdecay=1.2e-6, 
        bow_norm=1, 
        device=device, 
        train_embeddings=False, 
        embeddings_path=embeddings_file, 
        embeddings_type='pickle', 
        binary_embeddings=True, 
        headerless_embeddings=False, 
        use_partitions=True)
            

In [17]:
embs = pickle.load(open(embeddings_file, 'rb'))

In [18]:
print('Embeddings shape:', embs.shape)

Embeddings shape: (50, 768)


In [19]:
print(embs)

[[ 0.83585376 -0.25974375 -0.216797   ... -0.18789074 -0.49326912
  -0.9179299 ]
 [ 0.22354187  0.21200508  0.7886475  ... -0.47905082 -0.33619213
  -0.31313756]
 [ 1.3479398  -0.01886051 -0.16627687 ... -0.47807914  0.10327217
  -0.8390184 ]
 ...
 [ 0.72178626  0.86839515 -0.08751168 ... -0.17257741  0.34163907
  -0.7903824 ]
 [ 0.5833137   0.27970174  0.4332806  ...  0.0051777  -0.24891414
  -0.58246946]
 [ 0.4199069  -0.11670721 -0.3256204  ... -0.11589666 -0.6577412
  -0.7278601 ]]


In [20]:
output = model.train_model(dataset, top_words=20)

AttributeError: 'numpy.ndarray' object has no attribute 'split'

In [11]:
td, ch = TopicDiversity(topk=10), Coherence(topk=20, measure = 'c_v') # Initialize metric

In [ ]:
print("Coherence: ", ch.score(output))
print("Topic Diversity: ", td.score(output))